# **PINN for Coupled Spring-Mass System: A Guide**

This notebook serves as a guide to understanding and running the refactored PINN project. 

1.  **Part 1: Theoretical Background** provides a brief, high-level overview of the theory and mathematics implemented in the `src/` directory, as requested.
2.  **Part 2: Project Execution** provides the shell commands to install dependencies, train the model, and evaluate the results.

## Part 1: **Theoretical Background (A Guide to PINNs)**

A Physics-Informed Neural Network (PINN) is a neural network that solves differential equations by encoding the equation itself into the loss function. It doesn't require pre-solved data to train.

Our goal is to find the functions $x_i(t)$ for $i=1, ..., N$ that satisfy the system's physics.

### 1. **The Model**

The model is a simple feed-forward network (FFN) that takes a single input, time $t$, and outputs a vector of $N$ positions: $\mathbf{x}(t) = [x_1(t), x_2(t), ..., x_N(t)]$. 

$$
\text{NN}(t; \theta) \rightarrow \mathbf{x}(t)
$$

We can get the velocity $\dot{\mathbf{x}}(t)$ and acceleration $\ddot{\mathbf{x}}(t)$ by applying PyTorch's automatic differentiation (`torch.autograd.grad`) to the network's output $\mathbf{x}(t)$ with respect to its input $t$.

### 2. **The Loss Function**

The total loss, $L_{\text{total}}$, is a weighted sum of two components: the **physics loss** and the **initial condition loss**.

$$ 
L_{\text{total}} = w_{\text{physics}} L_{\text{physics}} + w_{\text{ic}} L_{\text{ic}} 
$$ 

--- 

#### **Physics Loss** ($L_{\text{physics}}$)

This loss ensures the model's predictions obey the laws of physics (our ODEs) at all times $t > 0$. We define a **residual** $f_i(t)$ for each mass $i$, which should be zero if the equation is satisfied.

The equation of motion for mass $i$ is:
$$ 
m\ddot{x}_i = k(x_{i+1} - x_i) - k(x_i - x_{i-1}) 
$$ 
$$ 
\ddot{x}_i = \frac{k}{m} (x_{i+1} - 2x_i + x_{i-1}) 
$$ 
Letting $\alpha = k/m$, the residual $f_i(t)$ is:

$$ 
f_i(t) = \ddot{x}_i(t) - \alpha (x_{i+1}(t) - 2x_i(t) + x_{i-1}(t)) 
$$ 

(with boundary conditions $x_0(t) = 0$ and $x_{N+1}(t) = 0$).

The loss is the Mean Squared Error (MSE) of these residuals over a batch of $N_{\text{physics}}$ random time points (collocation points) $t_j$ from $(0, T]$:

$$ 
L_{\text{physics}} = \frac{1}{N_{\text{physics}}} \sum_{j=1}^{N_{\text{physics}}} \sum_{i=1}^{N} \left( f_i(t_j) \right)^2 
$$ 

This is implemented in `src/physics_loss.py:physics_informed_loss`.

--- 

#### **Initial Condition Loss** ($L_{\text{ic}}$)

This loss ensures the model satisfies the known starting conditions at $t=0$. It is composed of a position loss and a velocity loss, evaluated *only* at $t=0$.

**1. Position Loss ($L_{\text{pos}}$):**
The initial positions are $\mathbf{x}(0) = [-x_0, 0, ..., 0]$.
$$ 
L_{\text{pos}} = \frac{1}{N} \sum_{i=1}^{N} (x_i(0) - x_i^{\text{target}}(0))^2 
$$ 

**2. Velocity Loss ($L_{\text{vel}}$):**
The initial velocities are $\dot{\mathbf{x}}(0) = [0, 0, ..., 0]$.
$$ 
L_{\text{vel}} = \frac{1}{N} \sum_{i=1}^{N} (\dot{x}_i(0) - \dot{x}_i^{\text{target}}(0))^2 
$$ 

The total IC loss is their sum: 
$$ 
L_{\text{ic}} = L_{\text{pos}} + L_{\text{vel}} 
$$ 

This is implemented in `src/physics_loss.py:initial_condition_loss` and is applied to a dedicated batch of $t=0$ points.

## Part 2: **Project Execution**

Now, let's run the project scripts.

### Step 1: **Install Dependencies**

In [ ]:
!pip install -r requirements.txt

### Step 2: **Train the Model**

We will run the `train.py` script. This will:
1.  Train the PINN model for 10,000 epochs (this may take a few minutes).
2.  Log all progress to `logs/pinn_train.log` and the console.
3.  Save the final trained model to `models/pinn_model.pth`.
4.  Save the loss curve plot to `plots/loss_curve.png`.

We use a higher weight (`--w_ic 10.0`) for the initial condition loss to ensure it is strongly enforced, which helps convergence.

In [ ]:
!python scripts/train.py \
    --num_epochs 10000 \
    --num_masses 3 \
    --num_train_physics 5000 \
    --num_train_ic 1000 \
    --learning_rate 1e-3 \
    --w_ic 10.0 \
    --log_freq 100

### Step 3: **Evaluate the Model**

Next, we run the `evaluate.py` script. This will:
1.  Load the saved model from `models/pinn_model.pth`.
2.  Solve the same ODE system using a traditional, high-precision numerical solver (`scipy.solve_ivp`).
3.  Generate and save a comparison plot to `plots/pinn_vs_ode_comparison.png`.

In [ ]:
!python scripts/evaluate.py \
    --num_masses 3

### Step 4: **View Results**

Let's display the plots generated by the scripts.

In [ ]:
from IPython.display import Image, display
import os

loss_plot = 'plots/loss_curve.png'
eval_plot = 'plots/pinn_vs_ode_comparison.png'

print("--- Training and Validation Loss ---")
if os.path.exists(loss_plot):
    display(Image(filename=loss_plot))
else:
    print(f"Plot not found at {loss_plot}")

print("\n--- PINN vs. ODE Solver Comparison ---")
if os.path.exists(eval_plot):
    display(Image(filename=eval_plot))
else:
    print(f"Plot not found at {eval_plot}")

### Step 5: **(Optional) View Logs**

You can inspect the detailed training log file.

In [ ]:
# Display the last 20 lines of the training log
!tail -n 20 logs/pinn_train.log